# Bank Customer Churn Analysis: A Data-Driven Retention Strategy

---

## Executive Summary

This analysis investigates the drivers of customer churn for an international bank operating in France, Germany, and Spain. By leveraging exploratory data analysis (EDA) and statistical testing, we identified key behavioral and demographic signals that predict customer departure.

**The Discovery:** We debunked the "more products is better" hypothesis, uncovering a **U-shaped risk curve** where customers with exactly 2 products are the most loyal. Customers outside this "sweet spot" (1, 3, or 4 products) exhibit significantly higher churn.

**The Outcome:** We engineered a composite **"Danger Score" (0-4)** that captures the intersection of the key risk factors. This score isolates a high-priority segment of **271 customers** with an **87%+ churn probability**, representing **$32.7 Million** in assets at risk.

---

## 1. Objective & Business Context

**The Problem:** Customer churn is a silent profit killer. Acquiring a new customer costs 5-25x more than retaining an existing one. This bank observed a global churn rate of ~20%, but lacks actionable insight into *who* is leaving and *why*.

**The Goal:** Move from a passive, reactive stance to a proactive, data-driven retention strategy by:
1.  Identifying the key features that differentiate churned customers from retained ones.
2.  Engineering a predictive "risk score" to flag high-probability churners *before* they leave.
3.  Providing actionable, targeted recommendations to the retention team.

In [ ]:
# --- Setup & Data Loading ---
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, ttest_ind

# Professional Aesthetics
sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.family'] = 'sans-serif'

# Load the curated dataset
df = pd.read_csv('data/Final_Portfolio_Dataset_v2.csv')
print(f"✅ Dataset loaded: {len(df):,} customer records.")
print(f"📊 Overall Churn Rate: {df['Exited'].mean():.2%}")

---

## 2. Exploratory Data Analysis (EDA): The Discovery Phase

Before building any model, we must understand what the data is telling us. EDA is about finding the *stories* in the numbers. Here, we uncover two critical anomalies that will inform our entire strategy.

### 2.1 The German Anomaly: A Regional Red Flag

Our first step is to look at churn by geography. Is the problem uniform, or are there regional hotspots?

In [ ]:
# --- Chart 1: The German Anomaly ---
plt.figure(figsize=(10, 6))
churn_by_geo = df.groupby('Geography')['Exited'].mean().sort_values(ascending=False).reset_index()
ax = sns.barplot(x='Geography', y='Exited', data=churn_by_geo, hue='Geography', palette='flare', legend=False)
plt.axhline(df['Exited'].mean(), color='#333333', linestyle='--', linewidth=1.5, label=f'Overall Avg: {df["Exited"].mean():.1%}')
plt.ylabel('Churn Rate', fontsize=12)
plt.xlabel('Geography', fontsize=12)
plt.title('Churn Rate by Geography', fontsize=14, fontweight='bold')
plt.legend(loc='upper right')
for i, row in churn_by_geo.iterrows():
    ax.text(i, row.Exited + 0.015, f'{row.Exited:.1%}', ha='center', fontsize=11, fontweight='bold')
plt.ylim(0, 0.45)
plt.tight_layout()
plt.show()

**💡 Data Insight:** Germany is experiencing a churn crisis. At **32.4%**, the German churn rate is nearly **double** that of France (16.2%) and Spain (16.7%). This is not statistical noise; it's a clear signal that something is fundamentally different about the German customer experience or market. This finding will become a core component of our risk score.

### 2.2 The U-Shaped Product Curve: Debunking "More is Better"

A common business assumption is that customers with more products are more "locked in" and thus more loyal. Let's test that hypothesis.

In [ ]:
# --- Chart 2: The U-Shaped Product Curve ---
plt.figure(figsize=(10, 6))
churn_by_product = df.groupby('NumOfProducts')['Exited'].mean().reset_index()
ax = sns.barplot(x='NumOfProducts', y='Exited', data=churn_by_product, hue='NumOfProducts', palette='rocket_r', legend=False)
plt.axhline(df['Exited'].mean(), color='#333333', linestyle='--', linewidth=1.5, label=f'Overall Avg: {df["Exited"].mean():.1%}')
plt.ylabel('Churn Rate', fontsize=12)
plt.xlabel('Number of Products', fontsize=12)
plt.title('Churn Rate by Number of Products (The U-Curve)', fontsize=14, fontweight='bold')
plt.legend(loc='upper left')
for i, row in churn_by_product.iterrows():
    ax.text(i, row.Exited + 0.02, f'{row.Exited:.1%}', ha='center', fontsize=11, fontweight='bold')
plt.ylim(0, 1.1)
plt.tight_layout()
plt.show()

**💡 Data Insight:** The data emphatically **rejects** the "more is better" hypothesis. Instead, we see a **U-shaped risk curve**:

| Products | Churn Rate | Interpretation |
|:--------:|:----------:|:---------------|
| **1** | ~28% | *"Low Engagement"*: The customer hasn't committed. They have one foot out the door. |
| **2** | ~8% | *"The Sweet Spot"*: The ideal level. Customers are engaged but not overwhelmed. |
| **3** | ~83% | *"Product Saturation"*: Something has gone wrong. Were they mis-sold? |
| **4** | **100%** | *"Departure Signal"*: Every single customer in this segment has churned. |

This insight is critical. Any customer with **1, 3, or 4** products is outside the safety zone and should be flagged as higher risk.

### 2.3 The Age Factor: Who is Leaving?

Let's explore if age plays a role in churn. Are we losing a specific demographic?

In [ ]:
# --- Chart 3: Age Distribution by Churn Status ---
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='Age', hue='Exited', kde=True, palette={0: '#2ecc71', 1: '#e74c3c'}, alpha=0.6, bins=30)
plt.axvline(45, color='black', linestyle='--', linewidth=1.5, label='Age Threshold (45)')
plt.title('Age Distribution: Churned vs. Retained Customers', fontsize=14, fontweight='bold')
plt.xlabel('Age', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.legend(title='Exited', labels=['Retained', 'Churned', 'Age 45 Threshold'])
plt.tight_layout()
plt.show()

**💡 Data Insight:** The distributions are strikingly different. Retained customers (green) skew younger, peaking in the early 30s. Churned customers (red) have a broader, older distribution, peaking in the mid-40s. A T-test confirms the mean age of churned customers is **7+ years older** than retained ones (p-value < 0.0001). Customers **aged 45+** represent a statistically significant risk group.

### 2.4 Statistical Validation: Proving it's Not Random

Our EDA has identified signals, but are they statistically significant? We use Chi-Square tests for categorical features and T-tests for numerical ones.

In [ ]:
# --- Statistical Tests ---
def chi_square_test(feature):
    table = pd.crosstab(df[feature], df['Exited'])
    chi2, p, dof, expected = chi2_contingency(table)
    return {'Feature': feature, 'Test': 'Chi-Square', 'P-Value': f"{p:.2e}", 'Significant': '✅ Yes' if p < 0.05 else '❌ No'}

def t_test(feature):
    churned = df[df['Exited'] == 1][feature]
    retained = df[df['Exited'] == 0][feature]
    t_stat, p = ttest_ind(churned, retained, equal_var=False)
    return {'Feature': feature, 'Test': 'T-Test', 'P-Value': f"{p:.2e}", 'Significant': '✅ Yes' if p < 0.05 else '❌ No'}

cat_features = ['Geography', 'Gender', 'NumOfProducts', 'IsActiveMember']
num_features = ['Age', 'Balance', 'CreditScore', 'Tenure', 'EstimatedSalary']

results = [chi_square_test(f) for f in cat_features] + [t_test(f) for f in num_features]
pd.DataFrame(results)

**💡 Data Insight:** Geography, Gender, NumOfProducts, IsActiveMember, Age, and Balance are all statistically significant predictors of churn. Tenure, CreditScore, and Salary are not. This confirms our EDA focus was on the right variables.

---

## 3. The Pivot: Engineering the "Danger Score"

The EDA identified multiple independent risk factors. The next step is to combine them into a single, actionable metric. We call this the **Danger Score**.

### The Logic

A customer earns **+1 point** for each risk factor present. The maximum score is 4.

| Factor | Condition | Rationale (from EDA) |
|:-------|:----------|:---------------------|
| **🇩🇪 German Anomaly** | Geography == 'Germany' | 2x higher churn rate than other regions. |
| **📦 Product Instability** | NumOfProducts in {1, 3, 4} | Outside the "2-product sweet spot"; U-curve risk. |
| **😴 Inactivity** | IsActiveMember == 0 | Disengaged customers are flight risks. |
| **👴 Age Sentinel** | Age >= 45 | Older demographic shows significantly higher churn. |

In [ ]:
# --- Chart 4: Danger Score Effectiveness ---
plt.figure(figsize=(10, 6))
churn_by_score = df.groupby('DangerScore').agg(
    ChurnRate=('Exited', 'mean'),
    Count=('Exited', 'count')
).reset_index()

ax = sns.barplot(x='DangerScore', y='ChurnRate', data=churn_by_score, hue='DangerScore', palette='YlOrRd', legend=False)
plt.ylabel('Churn Rate', fontsize=12)
plt.xlabel('Danger Score', fontsize=12)
plt.title('Churn Rate by Danger Score', fontsize=14, fontweight='bold')

for i, row in churn_by_score.iterrows():
    ax.text(i, row.ChurnRate + 0.03, f'{row.ChurnRate:.1%}\n(n={row.Count:,})', ha='center', fontsize=10)

plt.ylim(0, 1.1)
plt.tight_layout()
plt.show()

**💡 Data Insight:** The Danger Score is a powerful discriminator. **Churn rate scales almost linearly with the score.** The Score 4 segment, while small (271 customers), has an ~**87% churn rate**. These are the customers the retention team should prioritize immediately. This is the power of combining EDA insights into a composite metric.

---

## 4. Strategic Recommendations

Data without action is just trivia. Here's how the bank can deploy these findings:

### Targeted Plays

| Segment | Action | Rationale |
|:--------|:-------|:----------|
| **Danger Score 4** | **"VIP Rescue"**: Personal outreach from a senior relationship manager. Offer bespoke solutions. | These are near-certain churners. High-touch intervention is justified. |
| **1-Product Customers** | **"Ecosystem Entry"**: Incentivize adoption of a 2nd product (e.g., savings account with bonus rate). | Move them into the "sweet spot" to increase stickiness. |
| **German Market** | **"Regional Audit"**: Conduct qualitative research (surveys, interviews) to understand the root cause. Is it service quality? Competitor activity? | The data shows *what*, not *why*. We need to investigate. |
| **Age 45+ / Inactive** | **"Senior Digital Concierge"**: Offer personalized onboarding for digital tools. Proactive check-in calls. | Address potential friction with technology or feeling undervalued. |

### The Financial Stakes

The 271 customers in the Danger Score 4 segment hold an average balance of ~$120,000.

*   **Total Assets at Risk:** ~**$32.7 Million**
*   **Projected Savings (at 20% retention):** ~**$6.5 Million** protected from churn.

---

**Conclusion:** This analysis demonstrates how a structured, evidence-based approach to data science can transform raw records into strategic business value. The Danger Score is not just a number; it's a prioritization engine for the retention team.